# The Bottleneck Rule

- focus on bottlenecks
- not bigin too early
- begin with algorithms

# Compilation

In [ ]:
; ; declartion
; (declare (optimize (speed 3) (safety 0)))

; ; globaly
; (declaim (optimize (speed 3)
;                    (compilation-speed 0)
;                    (safety 0)
;                    (debug 0)))

In [2]:
; tail call
(defun length/r (lst)
  (if (null lst)
      0
      (1+ (length/r (cdr lst)))))

(defun length/tr (lst)
  (labels ((len (lst acc)
                (if (null lst)
                    acc
                    (len (cdr lst) (1+ acc)))))
    (len lst 0)))

(let ((lst '(a b c d)))
    (print (length/r lst))
    (print (length/tr lst)))

LENGTH/R

LENGTH/TR

4

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::LENGTH/R in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::LENGTH/TR in DEFUN

4 
4 

In [3]:
; inline
(declaim (inline single?))
(defun single? (lst)
  (and (consp lst) (null (cdr lst))))

(defun foo (x)
  (single? x))

NIL

SINGLE?

FOO

SB-INT:SIMPLE-STYLE-WARNING: DECLAIM where DECLARE was probably intended


In [4]:
; disassemble
(disassemble #'single?)

NIL


; disassembly for SINGLE?
; Size: 59 bytes. Origin: #xB80103F98F    ; SINGLE?
;   8F:  498B4510         MOV  RAX, [R13+16]   ; thread.binding-stack-pointer


;   93:  488945F0         MOV  [RBP-16], RAX

;   97:  4981F917010050   CMP  R9, #x50000117   ; NIL


;   9E:  740A             JEQ  L0

;   A0:  458D41F9         LEA  R8D, [R9-7]

;   A4:  41F6C00F         TEST  R8B, 15

;   A8:  7408             JEQ  L2

;   AA: L0:  BA17010050       MOV  EDX, #x50000117   ; NIL


;   AF: L1:  C9               LEAVE

;   B0:  F8               CLC

;   B1:  C3               RET

;   B2: L2:  4D8B5101         MOV  R10, [R9+1]

;   B6:  4981FA17010050   CMP  R10, #x50000117   ; NIL


;   BD:  75EB             JNE  L0

;   BF:  BA4F010050       MOV  EDX, #x5000014F   ; T


;   C4:  EBE9             JMP  L1

;   C6:  CC0F             INT3  15   ; Invalid argument count trap


;   C8:  CC0F             INT3  15   ; Invalid argument count trap



# Type Declarations

In [6]:
(declaim (type fixnum *count*))
(declaim (fixnum *count*)) ; omit 'type'

(defun ploy (a b x)
  (declare (fixnum a b x))
  (+ (* a (expt x 2)) (* b x)))


(*COUNT*)

(*COUNT*)

PLOY

SB-INT:SIMPLE-STYLE-WARNING: DECLAIM where DECLARE was probably intended
SB-INT:SIMPLE-STYLE-WARNING: DECLAIM where DECLARE was probably intended


In [7]:
; the: declare the value of an expression to be of a certain type
(defun poly (a b x)
  (declare (fixnum a b x))
  (the fixnum (+ (the fixnum (* a (the fixnum (expt x 2))))
                 (the fixnum (* b x)))))

POLY

In [8]:
; specialized array
(let ((x nil)
      (y nil))
    (setf x (vector 1.234d0 2.345d0 3.456d0)
        y (make-array 3 :element-type 'double-float)
        (aref y 0) 1.234d0
        (aref y 1) 2.345d0
        (aref y 2) 3.456d0))

3.456d0

SB-INT:SIMPLE-STYLE-WARNING: The variable X is assigned but never read.


In [12]:
; vector declaration
; (declare (type (vector fixnum 20) v))

In [14]:
; summing an array
(defun sum-elts (a)
  (let ((sum 0.0s0))
    (dotimes (r 1000)
      (dotimes (c 1000)
        (incf sum (aref a r c))))
    sum))

(defun sum-elts/t (a)
  (declare (type (simple-array single-float (1000 1000)) a))
  (let ((sum 0.0s0))
    (declare (type single-float sum))
    (dotimes (r 1000)
      (dotimes (c 1000)
        (incf sum (aref a r c))))
    sum))

(let ((a (make-array '(1000 1000)
           :element-type 'single-float
           :initial-element 1.0s0)))
  (time (sum-elts a))
  (time (sum-elts/t a)))

SUM-ELTS

SUM-ELTS/T

1000000.0

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SUM-ELTS in DEFUN

Evaluation took:
  0.015 seconds of real time
  0.015103 seconds of total run time (0.014915 user, 0.000188 system)
  100.00% CPU
  38,033,133 processor cycles
  0 bytes consed
  

Evaluation took:
  0.001 seconds of real time
  0.001506 seconds of total run time (0.001489 user, 0.000017 system)
  200.00% CPU
  3,665,002 processor cycles
  0 bytes consed
  


# Garbage Avoidance

In [21]:
; allocate block of memory beforehand
(setf *print-array* t)

(setf vec (make-array 10 :fill-pointer 2 :initial-element nil))
(length vec)
(vector-push 'a vec)
vec
(vector-pop vec)
vec

T

#(NIL NIL)

2

2

#(NIL NIL A)

A

#(NIL NIL)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC


In [22]:
; aref instead of svref
(setf (aref vec 0) 1)
(setf (aref vec 1) 2)
(vector-push 3 vec)
vec
; map-into instead of map
(setf vec (map-into vec #'1+ vec))
vec

1

2

2

#(1 2 3)

#(2 3 4)

#(2 3 4)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::VEC


In [1]:
; use large vector
(defconstant dict (make-array 25000 :fill-pointer 0))

(defun read-words (from)
  (setf (fill-pointer dict) 0)
  (with-open-file (in from :direction :input)
    (do ((w (read-line in nil :eof)
            (read-line in nil :eof)))
        ((eql w :eof))
      (vector-push w dict))))

(defun xform (fn seq) (map-into seq fn seq))

(defun write-words (to)
  (with-open-file (out to :direction :output
                       :direction :supersede)
    (map nil #'(lambda (x)
                 (fresh-line out)
                 (princ x out))
      (xform #'nreverse (sort (xform #'nreverse dict)
                            #'string<)))))

(read-words "sample.txt")
dict
(write-words "sample2.txt")

DICT

READ-WORDS

XFORM

WRITE-WORDS

NIL

#("The quick brown fox jumps over the lazy dog.")

NIL

In [2]:
; dynamic extent: avoid allocating space for temporary values on the heap

; rev: value last longer than the variable
(defun our-reverse (lst)
  (let ((rev nil))
    (dolist (x lst)
      (push x rev))
    rev))

; args
(defun our-adjoin (obj lst &rest args)
  (declare (dynamic-extent args))
  (if (apply #'member obj lst args)
      lst
      (cons obj lst)))

OUR-REVERSE

OUR-ADJOIN

# Example: Pools

In [1]:
; version 1
(defparameter *harbor* nil)

(defstruct ship
  name
  flag
  tons)

(defun enter (n f d)
  (push (make-ship :name n :flag f :tons d) *harbor*))

(defun find-ship (n)
  (find n *harbor* :key #'ship-name))

(defun leave (n)
  (setf *harbor*
    (delete (find-ship n) *harbor*)))

*HARBOR*

SHIP

ENTER

FIND-SHIP

LEAVE

In [2]:
; version 2
(defconstant pool (make-array 1000 :fill-pointer t))

(defstruct ship
  name
  flag
  tons)

(dotimes (i 1000)
  (setf (aref pool i) (make-ship)))

(defconstant harbor (make-hash-table :size 1100 :test #'eq))

(defun enter (n f d)
  (let ((s (if (plusp (length pool))
               (vector-pop pool)
               (make-ship))))
    (setf (ship-name s) n
      (ship-flag s) f
      (ship-tons s) d
      (gethash n harbor) s)))

(defun fund-ship (n) (gethash n harbor))

(defun leave (n)
  (let ((s (gethash n harbor)))
    (remhash n harbor)
    (vector-push s pool)))

POOL

SHIP

NIL

HARBOR

ENTER

FUND-SHIP

LEAVE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COPY-SHIP in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SHIP-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::SHIP-NAME) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SHIP-NAME in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::SHIP-FLAG) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SHIP-FLAG in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::SHIP-TONS) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SHIP-TONS in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-SHIP in DEFUN
SB-INT:CONSTANT-MODIFIED: Destructive function SB-KERNEL:%PUTHASH called on constant data: #<HASH-TABLE :TEST EQ :COUNT 0 {100415D643}>
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANS

# Fast Operators

In [3]:
(time (apply #'+ '(1 2 3)))
(time (reduce #'+ '(1 2 3)))

6

6


Evaluation took:
  0.000 seconds of real time
  0.000003 seconds of total run time (0.000002 user, 0.000001 system)
  100.00% CPU
  882 processor cycles
  0 bytes consed
  

Evaluation took:
  0.000 seconds of real time
  0.000002 seconds of total run time (0.000002 user, 0.000000 system)
  100.00% CPU
  573 processor cycles
  0 bytes consed
  


# Two-Phase Development

In [4]:
; see FFI(Foreign Function Interface) in SBCL `sb-alien` package.